# **IMAGES DATA RECOGNITION AND CATEGORIZATION BASED ON OCR**

---



---






# **Load Libraries**
---


In [ ]:
!pip install easyocr 
!python -m spacy download en_core_web_lg 
import easyocr 
import cv2 
import re
import spacy

# Load Spacy large model
nlp = spacy.load('en_core_web_lg') 
reader = easyocr.Reader(['en']) 

# **Define Functions**

---



In [ ]:
# select image format
def foramt(val):
  try:
    img = cv2.imread(val+".jpeg")
    cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
  except:
    try:
      img = cv2.imread(val+".jpg")
      cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    except:
      img = cv2.imread(val+".png")
      cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return img    

# Flatten list
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list  

# **API**

---




In [ ]:
# Enter image name
val = input("Enter Image name... : ")

# process image 
img = foramt(val)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
img = cv2.bilateralFilter(img,55, 75, 75) 
result = reader.readtext(img, detail = 0,paragraph=True,filter_ths=0.3, text_threshold=0.8,adjust_contrast=1.0)

# filter text
txt = re.sub(r"""[^a-zA-Z0-9+"*]""", ' ', str(result).upper())
price = []
price2 = []
company = []
company2 = []

# Classification of Price and Company name
doc = nlp(str(txt)) 
for ent in doc.ents:           
  if ent.label_ in ['CARDINAL','PERCENT','DATE','QUANTITY','MONEY'] and len(ent.text) > 1:
    price.append(ent.text)
  if ent.label_ in ['ORG','PRODUCT','PERSON','MONEY']:
    company.append(ent.text)


# Again classification of Company name for better results
doc = nlp(str(list(set(company))))
for ent in doc.ents:  
  if ent.label_ in ['ORG','PRODUCT','PERSON']:
    company2.append(ent.text)              
if len(company2) == 0:
  company2 = company

# Again classification of product price for better results
doc = nlp(str(price))
for ent in doc.ents: 
  if ent.label_ in ['CARDINAL','PRODUCT']:
    price2.append(ent.text)
if len(price2) == 0:
  price2 = price

# detail classification
Details=[]
# Repeate from starting with different filters
img = foramt(val)
img = cv2.GaussianBlur(img, (3, 3), 0)
img = cv2.bilateralFilter(img,55, 75, 75) 
result = reader.readtext(img, detail = 0,paragraph=True)

# You can select any of one case  

# Case >> 1
txt = re.sub(r"""[^a-zA-Z0-9+*]""", ' ', str(result))
doc = nlp(str(txt))
for ent in doc.ents: 
  if ent.label_ in ["WORK_OF_ART",'ORG','PERSON','LAW','PRODUCT','EVENT']:
    Details.append(ent.text) 
  # print(ent.text,ent.label_) 
if len(Details) == 0:
  Details.append(result)  

# Case >> 2
# Details.append(result)

# Transform list to 1D
info = flatten_list(Details)  
dict = {"Company Name":list(set(company2)),"Prices":list(set(price2)),"Product Details":list(info)} 
print("Company:",list(set(company2)),"||","Price",list(set(price2)),"||","Product Details",list(info),end=" ") 


Enter Image name... : 23
Company: ['ONEPLUS'] || Price ['256', '42 999'] || Product Details ['OnePlus 8T 5G', "8 + 128 GB '42,999", "12 + 256 GB '45,999", 'Buy Now'] 